# Hateful Speech Detection With Classic ML and Bert Models
### Description

In this notebook, I aim to explore various methods for detecting hateful speech within the dataset.
1. Translate Bangla hate speech into English 
1. TF-IDF with Linear Regression model
2. Bert model with native Pytorch 
3. Fine-tuning a model with the Trainer API

<a ></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Data Cleaning 💎</p>


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bengali-hate-speech-translated-to-english/translated_data.csv


In [ ]:
!pip install evaluate

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import string
from pathlib import Path

from collections import Counter
import random
import operator
from tqdm import tqdm
import time

from wordcloud import WordCloud
from string import punctuation
import nltk
import subprocess

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, BertTokenizer

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate

%matplotlib inline

In [ ]:
data_full=pd.read_csv('/kaggle/input/bengali-hate-speech-translated-to-english/translated_data.csv')
data_full.sample(10)

In [ ]:
data_full.drop(columns=['category'],inplace=True, axis=1)

In [ ]:
data_full.rename(columns={'hate':'Label','eng':'Content'},inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_full['Label'] = encoder.fit_transform(data_full['Label'])


In [ ]:
data_full = data_full.drop_duplicates(keep='first')
data_full.sample(5)

In [ ]:
data_full.groupby('Label').count()

In [ ]:
#The dataset is quite big so we take just 4000 examples of each class
df1 = data_full.query('Label == 0')
df2 = data_full.query('Label == 1')
data = pd.concat([df1, df2], ignore_index=True)

data.shape

<a ></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Data Exploration 💎</p>


In [ ]:
combined_title = ' '.join(df1['Content'])

wordcloud_img = WordCloud(width = 800, height = 800,
                            background_color ='white', colormap = 'BuGn',
                            min_font_size = 10).generate(combined_title)

plt.figure(figsize=(10,10))
plt.imshow(wordcloud_img)
plt.axis('off')
plt.title('Frequent words in Non-Hateful Comments')
plt.tight_layout(pad=2)
plt.show()

In [ ]:
combined_title = ' '.join(df2['Content'])

wordcloud_img = WordCloud(width = 800, height = 800,
                            background_color ='white', colormap = 'hot_r',
                            min_font_size = 10).generate(combined_title)

plt.figure(figsize=(10,10))
plt.imshow(wordcloud_img)
plt.axis('off')
plt.title('Frequent words in Hateful Comments')
plt.tight_layout(pad=2)
plt.show()

<a ></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Pre-processing 💎</p>


# TF-IDF with Linear Regression model  
  
This approach can be considered as baseline.  
Firstly, we need clean and prepare text: clean, remove stopwords, apply lemmatization OR stemming. By default I choose stemming.  
Then convert text to TF-IDF vector and use it as a feature for simple classification model LogisticRegression

In [ ]:
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

In [ ]:
data_tfidf = data.copy()

In [ ]:
stopwords_l = stopwords.words('english')

punctuation = re.compile("[" + re.escape(string.punctuation) + "]")

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english') #Snowball stemmer initialised

def text_cleaning(text, mode="stemming"):
    res = []
    text_clean = re.sub(punctuation,'',text)
    tokens = word_tokenize(text_clean)
    
    for token in tokens:
        if token.lower() not in stopwords_l:
            if mode == "stemming":
                prepared_word = stemmer.stem(token)
            else:
                prepared_word = lemmatizer.lemmatize(token)            
            res.append(prepared_word)
    return ' '.join(res)

In [ ]:
data_tfidf['cleaned_text'] = data_tfidf['Content'].apply(text_cleaning)

In [ ]:
data_tfidf.sample(5)

In [ ]:
train, test = train_test_split(data_tfidf, test_size=0.2, stratify=data_tfidf['Label'], random_state=42)

X_train = train['cleaned_text']
y_train = train['Label']

X_test = test['cleaned_text']
y_test = test['Label']

In [ ]:
# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X_train_vect = tfidf_vectorizer.fit_transform(X_train)

X_test_vect = tfidf_vectorizer.transform(X_test)

In [ ]:
linear_clf = LogisticRegression()

# train the model with training data processed using TF-IDF
linear_clf.fit(X_train_vect, y_train)

In [ ]:
y_pred_tf_idf = linear_clf.predict(X_test_vect)


report = classification_report(y_test, y_pred_tf_idf)
print(report)

display(pd.DataFrame({"Predicted: Unhateful": confusion_matrix(y_test, y_pred_tf_idf)[:, 0], 
              "Predicted: Hateful": confusion_matrix(y_test, y_pred_tf_idf)[:, 1]},
             index=['Actual: Unhateful', 'Actual: Hateful']))

<a ></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Bert model with native Pytorch  💎</p>

 
  
Now I want to train Bert model with classification head with native Pytorch.   


In [ ]:
data = data_tfidf

In [ ]:
data.head()

In [ ]:
train, validation = train_test_split(data, test_size=0.2, stratify=data_tfidf['Label'], random_state=42)

In [ ]:
# Define Dataset 
class HateSpeechDataset(Dataset):
    
    def __init__(self, data):
        
        # Initialize BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')        
        self.data = data
       
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data.iloc[idx]

        text = example["Content"]
        label = example["Label"]

        # Tokenize the text
        encoding = self.tokenizer.encode_plus(text, padding='max_length', truncation=True, max_length=64, return_tensors='pt')
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"], ##.unsqueeze(0).int(),
            "label": label,
        }

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')      

In [ ]:
dataset_train = HateSpeechDataset(train)
dataset_val = HateSpeechDataset(validation)

#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create DataLoader
batch_size = 128
dataloader_train = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    collate_fn=lambda x: {
        "input_ids": torch.stack([item["input_ids"] for item in x]),
        "attention_mask": torch.stack([item["attention_mask"] for item in x]),
        "labels": torch.tensor([item["label"] for item in x])
    },
    #collate_fn=data_collator,
    pin_memory=True,
)

dataloader_val = DataLoader(
    dataset_val,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    collate_fn=lambda x: {
        "input_ids": torch.stack([item["input_ids"] for item in x]),
        "attention_mask": torch.stack([item["attention_mask"] for item in x]),
        "labels": torch.tensor([item["label"] for item in x])
    },
    #collate_fn=data_collator,
    pin_memory=True,
)

In [ ]:
# Define BERT classifier
class BERTClassifier(nn.Module):
    
    def __init__(self):
        
        # Specify network layers
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        
        # Define dropout
        self.dropout = nn.Dropout(0.1)
        
        # Freeze BERT layers
        for n, p in self.bert.named_parameters():
            p.requires_grad = False
            
    def forward(self, text, masks):
        #output_bert = self.bert(text, attention_mask=masks).last_hidden_state.mean(axis=1)
        #print(output_bert.last_hidden_state)
        #print(self.bert.config.hidden_size)
        
        output_bert = self.bert(text, attention_mask=masks).last_hidden_state
        output_bert = self.avg_pool(output_bert.transpose(1, 2)).squeeze(-1)
        
        return self.linear(self.dropout(output_bert))

In [ ]:
model = BERTClassifier()


In [ ]:
# Define optimiser, objective function and epochs
optimizer = optim.Adam(model.parameters(), lr=0.000002) #optim.AdamW(model.parameters(), lr=5e-5) #
criterion = nn.BCEWithLogitsLoss()
epochs = 30

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

<a ></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Train with native Pytorch  💎</p>


In [ ]:
model.to(device)

val_losses = []
train_losses = []

# Train model
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================

    model.train()
    print(f"Start training epoch {epoch_i}...")
    total_train_loss = 0
    for i, batch in enumerate(tqdm(dataloader_train)):
    
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        masks = batch['attention_mask'].to(device)
        label = batch['labels'].to(device) 

        output = model(input_ids, masks)
        loss = criterion(output.squeeze(), label.float())

        loss.backward()
        
        # Clip the norm of the gradients to 1.0 to prevent the "exploding gradients".
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_train_loss += loss.item()
        
    avg_train_loss = total_train_loss / len(dataloader_train)
    train_losses.append(avg_train_loss)
    # ========================================
    #               Validation
    # ========================================
    
    model.eval()
    print("Start validation...")
    y_true_bert = list()
    y_pred_bert = list()
    
    total_eval_loss = 0.0
    with torch.no_grad():
        for batch in dataloader_val:
            input_ids = batch['input_ids'].to(device)
            masks = batch['attention_mask'].to(device)
            label = batch['labels'].to(device)
            
            output = model(input_ids, masks)
            max_output = (torch.sigmoid(output).cpu().numpy().reshape(-1)>= 0.5).astype(int)
            y_true_bert.extend(label.tolist())
            y_pred_bert.extend(max_output.tolist())
            
            loss_v = criterion(output.squeeze(), label.float())
            total_eval_loss += loss.item()
    avg_val_loss = total_eval_loss / len(dataloader_val)
    val_losses.append(avg_val_loss)
    
    print(f"Metrics after Epoch {epoch_i}")     
    print(f"Accuracy : {accuracy_score(y_true_bert, y_pred_bert)}")
    print(f"Presision: {np.round(precision_score(y_true_bert, y_pred_bert),3)}")
    print(f"Recall: {np.round(recall_score(y_true_bert, y_pred_bert),3)}") 
    print(f"F1: {np.round(f1_score(y_true_bert, y_pred_bert),3)}")
    print("   ")  

In [ ]:
print('Test accuracy: {:.2f}'.format(accuracy_score(y_true_bert, y_pred_bert)))
print('\nClassification report: \n', classification_report(y_true_bert, y_pred_bert))
print('\nConfusion matrix: \n')
display(pd.DataFrame({"Predicted: Unhateful": confusion_matrix(y_true_bert, y_pred_bert)[:, 0], 
              "Predicted: Hateful": confusion_matrix(y_true_bert, y_pred_bert)[:, 1]},
             index=['Actual: Unhateful', 'Actual: Hateful']))

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Validation Loss")
plt.plot(val_losses,label="val")
plt.plot(train_losses,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

<a ></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;"> Fine-tuning a model with the Trainer API  💎</p>
  
In this section, along with Trainer API, I apply the AutoModelForSequenceClassification. This model automatically includes a classification head, so there's no need for any additional configuration. Train params are default


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer", report_to="none")

In [ ]:
metric_acc = evaluate.load("accuracy")
metric_prec = evaluate.load("precision")
metric_recall = evaluate.load("recall")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    prec = metric_prec.compute(predictions=predictions, references=labels)["precision"]
    rec = metric_recall.compute(predictions=predictions, references=labels)["recall"]
    
    return {"accuracy": acc,"precision": prec, "recall": rec}

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')    
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    data_collator = data_collator
    
)

In [ ]:
trainer.train()

In [ ]:
model.eval()
print("Start validation...")
y_true_auto_bert  = list()
y_pred_auto_bert = list()

total_eval_loss = 0.0
with torch.no_grad():
    for batch in dataloader_val:
        input_ids = batch['input_ids'].to(device)
        masks = batch['attention_mask'].to(device)
        label = batch['labels'].to(device)

        output = model(input_ids, masks)
        max_output = np.argmax(output.logits.cpu().numpy(), axis=-1)
        y_true_auto_bert.extend(label.tolist())
        y_pred_auto_bert.extend(max_output.tolist())


print(f"Accuracy : {accuracy_score(y_true_auto_bert, y_pred_auto_bert)}")
print(f"Presision: {np.round(precision_score(y_true_auto_bert, y_pred_auto_bert),3)}")
print(f"Recall: {np.round(recall_score(y_true_auto_bert, y_pred_auto_bert),3)}") 
print(f"F1: {np.round(f1_score(y_true_auto_bert, y_pred_auto_bert),3)}")
print("   ")  


print('Test accuracy: {:.2f}'.format(accuracy_score(y_true_auto_bert, y_pred_auto_bert)))
print('\nClassification report: \n', classification_report(y_true_auto_bert, y_pred_auto_bert))
print('\nConfusion matrix: \n')
display(pd.DataFrame({"Predicted: Unhateful": confusion_matrix(y_true_auto_bert, y_pred_auto_bert)[:, 0], 
              "Predicted: Hateful": confusion_matrix(y_true_auto_bert, y_pred_auto_bert)[:, 1]},
             index=['Actual: Unhateful', 'Actual: Hateful']))